<div align="center">

# Оптимизация рабочих процессов с помощью конвейеров

</div>

---

**Pipeline (конвейер)** - инструмент из библиотеки **scikit-learn**, позволяющий обучить модель из произвольного кол-во шагов преобразования и применить ее для прогноза новых данных.

---

## Объединение преобразователей и оценивателей в конвейер

`Pipeline` в `scikit-learn` — это **последовательность шагов обработки данных и обучения модели**, объединённая в один объект.

* Каждый шаг, кроме последнего, — это **преобразователь** (например, масштабирование или PCA), который имеет методы `fit` и `transform`.
* Последний шаг — **модель (оцениватель)**, у которой есть метод `fit` для обучения и, обычно, `predict` для предсказаний.

Когда вы вызываете `pipeline.fit(X, y)`:

1. Данные проходят через первый преобразователь: он обучается (`fit`) на этих данных и преобразует их (`transform`).
2. Результат передаётся следующему преобразователю, который снова обучается и преобразует данные.
3. Так продолжается до последнего шага — модели, которая обучается уже на полностью подготовленных данных.

Плюсы такого подхода:

* Все этапы обработки и обучения объединены в один объект.
* Не нужно вручную повторять те же шаги для тестовой выборки — `Pipeline` делает это сам.
* Удобно использовать в кросс-валидации и при настройке гиперпараметров.

---

Набором данных для обучения будет набор данных о раке молочной железы из штата Висконсин (США) содержит 569 записей, каждая из которых описывает злокачественные (M — *malignant*) или доброкачественные (B — *benign*) опухолевые клетки.

* **Столбцы 1–2**: уникальный идентификатор образца и диагноз (M или B).
* **Столбцы 3–32**: 30 числовых признаков, рассчитанных на основе оцифрованных изображений ядер клеток. Эти признаки используются для построения модели, предсказывающей характер опухоли.

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [22]:
# Загрузка набора данных из локального хранилища
df = pd.read_csv('~/Рабочий стол/ML/Data/wdbc.data',
                 header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [23]:
# Разделяем данные на
X = df.loc[:, 2:].values
y = df.loc[:, 1].values

# Преобразуем метки классов (диагнозы) в целые числа
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

In [24]:
# Проверка сопоставлений le
le.transform(['M', 'B'])

array([1, 0])

In [25]:
# Разделение на train и test
X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                     test_size = 0.20,
                     stratify = y,
                     random_state = 1)


Объединение всё в конвейер **(`Pipeline`)**, чтобы обучение модели выполнялось полседовательно и единобразно для обучающей и тестовой выборке:
1. Провести **стандартизацию**, потому что значения в разных столбцах сильно различаются;
2. Сжать данные из 30-ого простарнства  в 2-ое подпространство с помощью PCA (метод гланых компонент);
3. Применить модель к преобразованным данным.

In [27]:
# Объединение всех этапов в конвейер
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components = 2),
                        LogisticRegression())

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
test_acc = pipe_lr.score(X_test, y_test)
print(f'Точность на тестовых данных: {test_acc:.3f}')

Точность на тестовых данных: 0.956
